In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import os

In [18]:
df = pd.read_csv("updated_dataset.csv")
df.head()

,index,website_name,competence_date,country_code,currency_code,brand,category1_code,category2_code,category3_code,product_code,title,itemurl,imageurl,full_price,price,full_price_eur,price_eur,flg_discount,local_image_path
0,100000,Farfetch,2023-11-17,MAC,HKD,MARQUES'ALMEIDA,CLOTHING,DRESSES,DAY DRESSES,19908319,Graffiti-print ruched mesh dress,http://www.farfetch.com/mo/shopping/women/marq...,https://cdn-images.farfetch-contents.com/19/90...,3852.0,3852.0,455.37,455.37,0,images/d19bc46963b0a3cba3b8bdf3cb2d0b67.jpg
1,100001,Farfetch,2023-11-17,MAC,HKD,VALENTINO GARAVANI,CLOTHING,PANTS,REGULAR-FIT & STRAIGHT LEG PANTS,19660972,Rockstud-embellished cargo trousers,http://www.farfetch.com/mo/shopping/men/valent...,https://cdn-images.farfetch-contents.com/19/66...,16800.0,10080.0,1986.05,1191.63,1,images/b32e184205b900e69ff0586c85d79181.jpg
2,100002,Farfetch,2023-11-17,MAC,HKD,ELISABETTA FRANCHI,CLOTHING,KNITS,KNITTED TOPS,21506295,roll neck crop top,http://www.farfetch.com/mo/shopping/women/elis...,https://cdn-images.farfetch-contents.com/21/50...,3571.0,3571.0,422.15,422.15,0,images/1fa7514996addde09e74b1839c51aa22.jpg
3,100003,Farfetch,2023-11-17,MAC,HKD,CASHMERE IN LOVE,CLOTHING,KNITS,KNITTED TROUSERS,12255754,cashmere Candiss flared knit trousers,http://www.farfetch.com/mo/shopping/women/cash...,https://cdn-images.farfetch-contents.com/12/25...,2556.0,2556.0,302.16,302.16,0,images/2bb91b55057cf2b16089e3e57c4a9708.jpg
4,100004,Farfetch,2023-11-17,MAC,HKD,HACKETT,CLOTHING,POLO SHIRTS,NaN,20191459,logo-embroidered cotton polo shirt,http://www.farfetch.com/mo/shopping/men/hacket...,https://cdn-images.farfetch-contents.com/20/19...,1538.0,1538.0,181.81,181.81,0,images/733796a55e7e83826c18a5c07d339b5e.jpg


In [19]:
cols = ['website_name', 'competence_date', 'country_code', 'currency_code',
       'brand', 'category2_code', 'category3_code',
       'product_code', 'title', 'itemurl', 'full_price', 'price',
       'flg_discount', 'local_image_path']

df = df[cols]
df.head()

,website_name,competence_date,country_code,currency_code,brand,category2_code,category3_code,product_code,title,itemurl,full_price,price,flg_discount,local_image_path
0,Farfetch,2023-11-17,MAC,HKD,MARQUES'ALMEIDA,DRESSES,DAY DRESSES,19908319,Graffiti-print ruched mesh dress,http://www.farfetch.com/mo/shopping/women/marq...,3852.0,3852.0,0,images/d19bc46963b0a3cba3b8bdf3cb2d0b67.jpg
1,Farfetch,2023-11-17,MAC,HKD,VALENTINO GARAVANI,PANTS,REGULAR-FIT & STRAIGHT LEG PANTS,19660972,Rockstud-embellished cargo trousers,http://www.farfetch.com/mo/shopping/men/valent...,16800.0,10080.0,1,images/b32e184205b900e69ff0586c85d79181.jpg
2,Farfetch,2023-11-17,MAC,HKD,ELISABETTA FRANCHI,KNITS,KNITTED TOPS,21506295,roll neck crop top,http://www.farfetch.com/mo/shopping/women/elis...,3571.0,3571.0,0,images/1fa7514996addde09e74b1839c51aa22.jpg
3,Farfetch,2023-11-17,MAC,HKD,CASHMERE IN LOVE,KNITS,KNITTED TROUSERS,12255754,cashmere Candiss flared knit trousers,http://www.farfetch.com/mo/shopping/women/cash...,2556.0,2556.0,0,images/2bb91b55057cf2b16089e3e57c4a9708.jpg
4,Farfetch,2023-11-17,MAC,HKD,HACKETT,POLO SHIRTS,NaN,20191459,logo-embroidered cotton polo shirt,http://www.farfetch.com/mo/shopping/men/hacket...,1538.0,1538.0,0,images/733796a55e7e83826c18a5c07d339b5e.jpg


In [20]:
df.isna().sum()

website_name            0
competence_date         0
country_code            0
currency_code           0
brand                   0
category2_code          0
category3_code      10373
product_code            0
title                   0
itemurl                 0
full_price              0
price                   0
flg_discount            0
local_image_path        5
dtype: int64

In [21]:
df['category3_code'] = df['category3_code'].fillna(" ")
df = df.dropna(subset="local_image_path")
df.isna().sum()

website_name        0
competence_date     0
country_code        0
currency_code       0
brand               0
category2_code      0
category3_code      0
product_code        0
title               0
itemurl             0
full_price          0
price               0
flg_discount        0
local_image_path    0
dtype: int64

# convert HKD into USD

In [6]:
def hkd_to_usd(price_hkd, rate=0.128):
    return price_hkd * rate

df['full_price'] = df['full_price'].apply(hkd_to_usd)
df['price'] = df['price'].apply(hkd_to_usd)

# Price Clusters

In [7]:
from sklearn.cluster import KMeans
import numpy as np

category_col = "category2_code"
price_col = "price"

def cluster_prices(group):
    prices = group[price_col].values.reshape(-1, 1)
    
    if len(prices) < 3:
        return pd.Series(["mid"] * len(prices), index=group.index)
    
    kmeans = KMeans(n_clusters=3, random_state=42, n_init="auto")
    labels = kmeans.fit_predict(prices)
    
    cluster_order = np.argsort(kmeans.cluster_centers_.flatten())
    label_map = {cluster_order[0]: "cheap", 
                 cluster_order[1]: "mid", 
                 cluster_order[2]: "luxury"}
    
    return pd.Series([label_map[l] for l in labels], index=group.index)

In [8]:
df["price_category"] = df.groupby(category_col, group_keys=False).apply(cluster_prices)

C:\Users\abdel\AppData\Local\Temp\ipykernel_23756\4112865025.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df["price_category"] = df.groupby(category_col, group_keys=False).apply(cluster_prices)


In [21]:
df.head()

,website_name,competence_date,country_code,currency_code,brand,category2_code,category3_code,product_code,title,itemurl,full_price,price,flg_discount,local_image_path,price_category
0,Farfetch,2023-11-17,MAC,HKD,PESERICO,KNITS,KNITTED SWEATERS,21755241,cable-knit wool-blend polo jumper,http://www.farfetch.com/mo/shopping/women/pese...,605.056,605.056,0,images/74c3930c3d5e3c8c804d8d6f3e59549d.jpg,cheap
1,Farfetch,2023-11-17,MAC,HKD,PHILOSOPHY DI LORENZO SERAFINI,DRESSES,EVENING DRESSES,19535310,pattern-print short-sleeve gown,http://www.farfetch.com/mo/shopping/women/phil...,1156.864,694.144,1,images/655f5b5e3588cc4b9549c42dac890542.jpg,cheap
2,Farfetch,2023-11-17,MAC,HKD,ALEMAIS,DRESSES,DAY DRESSES,20192671,Elisabetta floral-print linen minidress,http://www.farfetch.com/mo/shopping/women/alem...,489.088,489.088,0,images/2c43ce060f65d44fe153168975743d40.jpg,cheap
3,Farfetch,2023-11-17,MAC,HKD,PAULA,TOPS,T-SHIRTS & JERSEY SHIRTS,18003736,stripe-print linen T-shirt,http://www.farfetch.com/mo/shopping/women/paul...,104.960,104.960,0,images/49262d2bd3034d3f6ff740ad5df6afc5.jpg,cheap
4,Farfetch,2023-11-17,MAC,HKD,JIL SANDER,SWEATSHIRTS & KNITWEAR,KNITTED SWEATERS,20367745,decorative-buttons high-neck wool jumper,http://www.farfetch.com/mo/shopping/men/jil-sa...,1176.448,1176.448,0,images/8e59f1edde400bc0a78cf99ef71d0aa1.jpg,mid


In [ ]:
df.to_csv("ds.csv", index=False)